<a href="https://colab.research.google.com/github/Sandeephm/GoogleColab-codes/blob/master/CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-9bysd5z7
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-9bysd5z7
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=a64a6042edde1a3e8215bd8b362baea32a5a22090e1a9d10ead20d6296012266
  Stored in directory: /tmp/pip-ephem-wheel-cache-g88dlb0p/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu 
#include <iostream> 
int main() 
{ 
    std::cout << "Welcome To GeeksforGeeks\n"; 
    return 0; 
} 

Welcome To GeeksforGeeks



In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
__global__ void add(int *a, int *b, int *c) {
*c = *a + *b;
}
int main() {
int a, b, c;
// host copies of variables a, b & c
int *d_a, *d_b, *d_c;
// device copies of variables a, b & c
int size = sizeof(int);
// Allocate space for device copies of a, b, c
cudaMalloc((void **)&d_a, size);
cudaMalloc((void **)&d_b, size);
cudaMalloc((void **)&d_c, size);
// Setup input values  
c = 0;
a = 3;
b = 5;
// Copy inputs to device
cudaMemcpy(d_a, &a, size, cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, &b, size, cudaMemcpyHostToDevice);
// Launch add() kernel on GPU
add<<<1,1>>>(d_a, d_b, d_c);
// Copy result back to host
cudaError err = cudaMemcpy(&c, d_c, size, cudaMemcpyDeviceToHost);
  if(err!=cudaSuccess) {
      printf("CUDA error copying to Host: %s\n", cudaGetErrorString(err));
  }
printf("result is %d\n",c);
// Cleanup
cudaFree(d_a);
cudaFree(d_b);
cudaFree(d_c);
return 0;
}


result is 8



In [ ]:
%%cu 
#include <cstdio> 
#include <iostream> 
  
using namespace std; 
  
__global__ void maxi(int* a, int* b, int n) 
{ 
    int block = 256 * blockIdx.x; 
    int max = 0; 
  
    for (int i = block; i < min(256 + block, n); i++) { 
  
        if (max < a[i]) { 
            max = a[i]; 
        } 
    } 
    b[blockIdx.x] = max; 
} 
  
int main() 
{ 
  
    int n; 
    n = 3 >> 2; 
    int a[n]; 
  
    for (int i = 0; i < n; i++) { 
        a[i] = rand() % n; 
        cout << a[i] << "\t"; 
    } 
  
    cudaEvent_t start, end; 
    int *ad, *bd; 
    int size = n * sizeof(int); 
    cudaMalloc(&ad, size); 
    cudaMemcpy(ad, a, size, cudaMemcpyHostToDevice); 
    int grids = ceil(n * 1.0f / 256.0f); 
    cudaMalloc(&bd, grids * sizeof(int)); 
  
    dim3 grid(grids, 1); 
    dim3 block(1, 1); 
  
    cudaEventCreate(&start); 
    cudaEventCreate(&end); 
    cudaEventRecord(start); 
  
    while (n > 1) { 
        maxi<<<grids, block>>>(ad, bd, n); 
        n = ceil(n * 1.0f / 256.0f); 
        cudaMemcpy(ad, bd, n * sizeof(int), cudaMemcpyDeviceToDevice); 
    } 
  
    cudaEventRecord(end); 
    cudaEventSynchronize(end); 
  
    float time = 0; 
    cudaEventElapsedTime(&time, start, end); 
  
    int ans[2]; 
    cudaMemcpy(ans, ad, 4, cudaMemcpyDeviceToHost); 
  
    cout << "The maximum element is : " << ans[0] << endl; 
  
    cout << "The time required : "; 
    cout << time << endl; 
} 

The maximum element is : 2015486880
The time required : 0.003328



Hello World Program in CUDA

In [ ]:
%%cu
#include <cstdio>
#include "cuda.h"

void CPUFunction()
{
  printf("Runs on the CPU.\n");
}

__global__
void GPUFunction()
{
  printf("Runs on the GPU.\n");
}

int main()
{
  CPUFunction();

  GPUFunction<<<1, 1>>>();

  cudaDeviceSynchronize();

  return EXIT_SUCCESS;
}

Runs on the CPU.
Runs on the GPU.



In [ ]:
%%cu
#include <iostream>
#include <math.h>
// Kernel function to add the elements of two arrays
__global__
void add(int n, float *x, float *y)
{
  for (int i = 0; i < n; i++)
    y[i] = x[i] + y[i];
}

int main(void)
{
  int N = 1<<20;
  float *x, *y;

  // Allocate Unified Memory – accessible from CPU or GPU
  cudaMallocManaged(&x, N*sizeof(float));
  cudaMallocManaged(&y, N*sizeof(float));

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the GPU
  add<<<1, 1>>>(N, x, y);

  // Wait for GPU to finish before accessing on host
  cudaDeviceSynchronize();

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  cudaFree(x);
  cudaFree(y);
  
  return 0;
}

Max error: 0



Dynamic Memmory Allocation

In [ ]:
%%cu
#include <stdio.h>

__global__ void dynamicReverse(int *d, int n)
{
  // Dynamic shared memory	
  extern __shared__ int s[];
  int t = threadIdx.x;
  int tr = n-t-1;
  s[t] = d[t];
  __syncthreads();
  d[t] = s[tr];
}

int main(void)
{
  const int n = 64;
  int a[n], r[n], d[n];

  for (int i = 0; i < n; i++) {
    a[i] = i;
    r[i] = n-i-1;
    d[i] = 0;
  }

  int *d_d;
  cudaMalloc(&d_d, n * sizeof(int)); 

  // run dynamic shared memory version
  cudaMemcpy(d_d, a, n*sizeof(int), cudaMemcpyHostToDevice);
  dynamicReverse<<<1,n,n*sizeof(int)>>>(d_d, n);
  cudaMemcpy(d, d_d, n * sizeof(int), cudaMemcpyDeviceToHost);
  for (int i = 0; i < n; i++) 
    if (d[i] != r[i]) printf("Error: d[%d]!=r[%d] (%d, %d)n", i, i, d[i], r[i]);
}

Run Hello 5 times

In [ ]:
%%cu
#include <iostream>
#include "cuda.h"

void helloCPU()
{
  std::cout<<"Hello from CPU.\n";
}

__global__ void helloGPU()
{
  printf("Hello also from GPU.\n");
}

int main()
{

  helloCPU();
  helloGPU<<<1,5>>>();
  cudaDeviceSynchronize();

  return EXIT_SUCCESS;
}


Hello from CPU.
Hello also from GPU.
Hello also from GPU.
Hello also from GPU.
Hello also from GPU.
Hello also from GPU.



In [ ]:
%%cu
#include <iostream>
#include "cuda.h"

void helloCPU()
{
  std::cout<<"Hello from CPU.\n";
}

__global__ void helloGPU()
{
  printf("Hello also from GPU.\n");
}

int main()
{

  helloCPU();
  helloGPU<<<5,5>>>();
  cudaDeviceSynchronize();

  return EXIT_SUCCESS;
}


Hello from CPU.
Hello also from GPU.
Hello also from GPU.
Hello also from GPU.
Hello also from GPU.
Hello also from GPU.
Hello also from GPU.
Hello also from GPU.
Hello also from GPU.
Hello also from GPU.
Hello also from GPU.
Hello also from GPU.
Hello also from GPU.
Hello also from GPU.
Hello also from GPU.
Hello also from GPU.
Hello also from GPU.
Hello also from GPU.
Hello also from GPU.
Hello also from GPU.
Hello also from GPU.
Hello also from GPU.
Hello also from GPU.
Hello also from GPU.
Hello also from GPU.
Hello also from GPU.



In [ ]:
%%cu
#include <stdio.h>
#include "cuda.h"

__global__ void printSuccessForCorrectExecutionConfiguration()
{
  if (threadIdx.x == 1023 && blockIdx.x == 255)
  {
    printf("Success!\n");
  }
}


int main()
{
  printSuccessForCorrectExecutionConfiguration<<<256,1024>>>();
  cudaDeviceSynchronize();
  return EXIT_SUCCESS;
}

Success!



In [ ]:
%%cu
#include <stdio.h>
#include "cuda.h"
__global__ void loop(int N)
{
  for (int i = 0; i < N; ++i)
  {
    printf("This is iteration number %d\n", i);
  }
}

int main()
{
  /*
   * When refactoring 'loop' to launch as a kernel, be sure
   * to use the execution configuration to control how many
   * "iterations" to perform.
   *
   * Use 1 block of threads.
   */

  int N = 10;
  loop<<<1,3>>>(N);
  cudaDeviceSynchronize();
  return EXIT_SUCCESS;
}

This is iteration number 0
This is iteration number 0
This is iteration number 0
This is iteration number 1
This is iteration number 1
This is iteration number 1
This is iteration number 2
This is iteration number 2
This is iteration number 2
This is iteration number 3
This is iteration number 3
This is iteration number 3
This is iteration number 4
This is iteration number 4
This is iteration number 4
This is iteration number 5
This is iteration number 5
This is iteration number 5
This is iteration number 6
This is iteration number 6
This is iteration number 6
This is iteration number 7
This is iteration number 7
This is iteration number 7
This is iteration number 8
This is iteration number 8
This is iteration number 8
This is iteration number 9
This is iteration number 9
This is iteration number 9



In [ ]:
%%cu
#include <cstdio>

/*
 * Initialize array values on the host.
 */

//Function to initialize an array
__global__ void init(int *a, int N)
{
  for (int i = 0; i < N; ++i) {
    a[i] = i;
  }
}

/*
 * Double elements in parallel on the GPU.
 */

__global__ void doubleElements(int *a, int N)
{
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  if (i < N) {
    a[i] *= 2;
  }
}

/*
 * Check all elements have been doubled on the host.
 */

bool checkElementsAreDoubled(int *a, int N)
{
  for (int i = 0; i < N; ++i) {
    if (a[i] != i*2) {
      return false;
    }
  }
  return true;
}

int main()
{
  int N = 100;
  int *a;
  size_t size = N * sizeof(int);
  
  cudaMallocManaged(&a, size); //allocate space for pointer a
  init<<<1,1>>>(a, N);         //initialize a pointer a

  size_t threads_per_block = 10;
  size_t number_of_blocks = 10;

  doubleElements<<<number_of_blocks, threads_per_block>>>(a, N);
  cudaDeviceSynchronize();

  bool areDoubled = checkElementsAreDoubled(a, N);
  printf("All elements were doubled? %s\n", areDoubled ? "TRUE" : "FALSE");

  cudaFree(a); //Free memory on GPU
}

All elements were doubled? TRUE



In [ ]:
%%cu
#include <cstdio>

void init(int *a, int N)
{
  for (int i = 0; i < N; ++i) {
    a[i] = i;
  }
}

/*
 * In the current application, 'N' is larger than the grid.
 * Refactor this kernel to use a grid-stride loop in order that
 * each parallel thread work on more than one element of the array.
 */

__global__
void doubleElements(int *a, int N)
{
  int indexWithinTheGrid = blockIdx.x * blockDim.x + threadIdx.x;
  int gridStride = gridDim.x * blockDim.x;
  for (int i = indexWithinTheGrid; i < N; i += gridStride)
  {
    a[i] *= 2;
  }  
}

bool checkElementsAreDoubled(int *a, int N)
{
  for (int i = 0; i < N; ++i) {
    if (a[i] != i*2) {
      return false;
    }
  }
  return true;
}

int main()
{
  /*
   * 'N' is greater than the size of the grid (see below).
   */

  int N = 10000;
  int *a;

  size_t size = N * sizeof(int);
  cudaMallocManaged(&a, size);

  init(a, N);

  size_t threads_per_block = 256;
  size_t number_of_blocks = 32;

  doubleElements<<<number_of_blocks, threads_per_block>>>(a, N);
  cudaDeviceSynchronize();

  bool areDoubled = checkElementsAreDoubled(a, N);
  printf("All elements were doubled? %s\n", areDoubled ? "TRUE" : "FALSE");

  cudaFree(a);
}

All elements were doubled? TRUE



In [ ]:
%%cu
#include <stdio.h>
#include "cuda.h"

/*
 * Host function to initialize vector elements. This function
 * simply initializes each element to equal its index in the
 * vector.
 */

void initWith(float num, float *a, int N)
{
  for(int i = 0; i < N; ++i) {
    a[i] = num;
  }
}

/*
 * Device kernel stores into 'result' the sum of each
 * same-indexed value of 'a' and 'b'.
 */

__global__
void addVectorsInto(float *result, float *a, float *b, int N)
{
  int index = threadIdx.x + blockIdx.x * blockDim.x;
  int stride = blockDim.x * gridDim.x;

  for(int i = index; i < N; i += stride) {
    result[i] = a[i] + b[i];
  }
}

/*
 * Host function to confirm values in 'vector'. This function
 * assumes all values are the same 'target' value.
 */

void checkElementsAre(float target, float *vector, int N)
{
  for(int i = 0; i < N; i++) {
    if(vector[i] != target)
    {
      printf("FAIL: vector[%d] - %0.0f does not equal %0.0f\n", i, vector[i], target);
      exit(1);
    }
  }
  printf("Success! All values calculated correctly.\n");
}

int main()
{
  const int N = 2<<24;
  size_t size = N * sizeof(float);

  float *a, *b, *c;

  cudaMallocManaged(&a, size);
  cudaMallocManaged(&b, size);
  cudaMallocManaged(&c, size);

  initWith(3, a, N);
  initWith(4, b, N);
  initWith(0, c, N);

  size_t threadsPerBlock;
  size_t numberOfBlocks;

  threadsPerBlock = 1;
  numberOfBlocks = 1;

  addVectorsInto<<<numberOfBlocks, threadsPerBlock>>>(c, a, b, N);

  cudaDeviceSynchronize();

  checkElementsAre(7, c, N);

  cudaFree(a);
  cudaFree(b);
  cudaFree(c);
}

Success! All values calculated correctly.

